If using colab, install necessary libraries

In [1]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
from urlextract import URLExtract as extract
extractor = extract()

In [39]:
# BFE Abnahme
packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,6cf5d98c-e259-4c40-a54b-a93b539eb79b,{'fr': 'Réseaux thermiques: offre de chaleur o...,https://www.bfe.admin.ch/waerme-und-kaelteangebot,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Text (.csv), comma..."
1,70faeac6-d660-492d-bc5b-dfdfa707007e,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
2,0a6ef29d-86b6-416f-99c3-5e6d26f1f506,{'fr': 'Situation et domaines attenants confor...,https://www.bfe.admin.ch/konsultationsbereiche...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'OGC:WMS', 'cache_last_updated':..."
3,46ee8db4-c4e9-436a-ad39-bb67102bffec,{'fr': 'Plan sectoriel des lignes de transport...,https://www.bfe.admin.ch/sp-uebertragungsleitu...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
4,bb13b62d-6cd1-4602-acc1-bbe2aba197b0,{'fr': 'Réseaux thermiques: demande concernant...,https://www.bfe.admin.ch/waerme-kaeltenachfrag...,[{'url': 'https://www.geocat.ch/geonetwork/srv...,"[{'protocol': 'WWW:DOWNLOAD:Geopackage (ogc)',..."
...,...,...,...,...,...
100,a37cb9ae-50f9-4fa5-a48c-95e2c5f0625b,{'fr': 'Aptitude des toitures à utiliser l’éne...,https://www.bfe.admin.ch/bfe/de/home/versorgun...,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
101,979f37ed-fcb5-4400-802b-be0bb2833ea4,{'fr': 'Statistiques heddomadaires: production...,http://www.bfe.admin.ch/ogd51,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
102,73d41982-f474-409a-a129-418678921128,{'fr': 'Etude sur l'électricité hivernale d'or...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.bfe.admin.ch/bfe/de/home...,"[{'byte_size': 5819680686, 'cache_last_updated..."
103,e839fa78-ba22-4772-be16-0843b99b7161,{'fr': 'Résultats de la modélisation PE2050+ :...,https://www.bfe.admin.ch/bfe/de/home/politik/e...,[{'url': 'https://www.uvek-gis.admin.ch/BFE/st...,"[{'byte_size': 0, 'cache_last_updated': None, ..."


---------------------------------------------------------------------------

Functions

In [4]:
def check_url(url):
    try:
        time.sleep(1)
        # head or get --> head is not working consistantly
        
        # redirect not working on every machine and not everytime... but this link technically works
        if url == 'https://bfe.admin.ch/shared-mobility-angebote':
            return True
        
        get = requests.get(url)
        
        if get.status_code < 400:
            return True
        else:
            return False
        
    except requests.exceptions.RequestException as e:
        print(e)
        raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [5]:
def addHTTPS(url):
    adjusted = url.replace("http:", "https:" )
    adjusted = adjusted.replace("www", "https://www" )
    
    return adjusted.replace("https://https://www", "https://www" )

In [6]:
def addHTTP(url):
    adjusted = url.replace("www", "http://www")
    return adjusted.replace("http://http://www", "http://www")

In [10]:
def getURLs(original_string, urls):
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = check_url(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = check_url(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = check_url(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks

    return links

In [14]:
def checkString(string):
  
  urls = extractor.find_urls(string)
  
  # first remove elements which are not actual links
  urls = [url for url in urls if ('http' in url) or ('www' in url)]

  # if the same url appears multiple times
  urls = list(set(urls))
  
  return getURLs(string, urls)

In [15]:
urls = ['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']
test = getURLs(str(urls), urls)

In [16]:
for i in test:
  print(i)

["['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']", "['https://https://www.google.ch', 'https://https://www.google.ch', 'https://www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']"]
['https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip', '-']


In [18]:
check_url('https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl')

True

------------------------------------------------------------------------------------------------------------------------------------------------------

For _url_ and _relations_

In [41]:
cols = ['url', 'relations']

completeList = []

for index, row in df.iterrows():
    
    print(index)
    dataset_name = row['title']['de']
    id           = row['id']
    
    for col in cols:
  
        att_type = 'dataset'
        att      = col
        val      = row[col]

        liste = []

        if col == 'resources':
            
            att_type = 'resources'
            #change att
            print('nichts')


        else:
            if type(val) == str:
                liste = checkString(val)
            else:
                liste = checkString(str(val))


        if len(liste) == 0:
            continue
        else:
            for elem in liste:
                completeList.append([dataset_name, att_type, att, id, elem[0], elem[1]])


0
['https://www.bfe.admin.ch/waerme-und-kaelteangebot']
['https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/b6c8d9c6-a2ca-435a-af0a-6a8ac94199fc']
1
['https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl']


KeyboardInterrupt: ignored

In [ ]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

In [ ]:
f.to_csv('urlAndRelations.csv', index=False)

-------------------------------------------------------------------------------------------------------------------------------------------------------------

For _resources_

In [42]:
# import module
from urllib.request import urlopen
from urllib.error import *
 
# try block to read URL

def checkLink(link):

  try:
      html = urlopen(link)
      
  # except block to catch
  # exception
  # and identify error
  except HTTPError as e:
      print("HTTP error", e)
      print(link)
      print()
      return False
      
  except URLError as e:
      print("Opps ! Page not found!", e)
      print(link)
      print()
      return False
  
  else:
      return True

In [57]:
def getURLs(original_string, urls):

    print(urls)
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = checkLink(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = checkLink(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = checkLink(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks


    

    return links

In [76]:
def checkDictionary(l):

  resourceList = []
  
  for d in l:
    print('Dictionary:')
    print(d)

    id = d['id']

    print('id: ', id)
    print('package_id: ', d['package_id'])

    for key in d.keys():

      #print(d)

      value    = d[key]
      isString = True

      if type(value) != str:
        isString = False
        value    = str(d[key])


      urls = extractor.find_urls(value)
  
      # first remove elements which are not actual links
      urls = [url for url in urls if ('http' in url) or ('www' in url)]

      whatToChange = []

      if len(urls) != 0:
        whatToChange = getURLs(value, urls)
      
      for elem in whatToChange:
        resourceList.append([key] + [id] + elem)

  return resourceList

In [68]:
d = df.iloc[40]['resources'].copy()
res = checkDictionary(d)

[{'byte_size': 0, 'protocol': 'OGC:WMS', 'cache_last_updated': None, 'issued': '2016-02-25T00:00:00', 'package_id': 'a7e83e99-3ee9-4ffc-b1d5-8a792537a161', 'datastore_contains_all_records_of_source_file': False, 'datastore_active': False, 'id': 'fbefbaa8-fcd7-4030-abdc-a38641febbf5', 'size': None, 'display_name': {'fr': 'ch.bfe.windenergie-geschwindigkeit_h100', 'de': 'ch.bfe.windenergie-geschwindigkeit_h100', 'en': 'ch.bfe.windenergie-geschwindigkeit_h100', 'it': 'ch.bfe.windenergie-geschwindigkeit_h100'}, 'title': {'fr': 'ch.bfe.windenergie-geschwindigkeit_h100', 'de': 'ch.bfe.windenergie-geschwindigkeit_h100', 'en': 'ch.bfe.windenergie-geschwindigkeit_h100', 'it': 'ch.bfe.windenergie-geschwindigkeit_h100'}, 'state': 'active', 'hash': '', 'description': {'fr': 'Vitesse du vent à 100 m du sol', 'de': 'Windgeschwindigkeit 100 m Höhe', 'en': 'Wind speed 100 metres above ground', 'it': 'Velocità vento a 100 m dal suolo'}, 'format': 'WMS', 'mimetype_inner': None, 'url_type': None, 'mimety

In [64]:
for r in res:
  print(r)

['download_url', 'f0904fec-5bb3-4189-bf2f-4d7bd4a560ab', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100_2056.fgdb.zip', '-']
['url', 'f0904fec-5bb3-4189-bf2f-4d7bd4a560ab', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100/windenergie-geschwindigkeit_h100_2056.fgdb.zip', '-']


In [78]:
completeList = []

for index, row in df.iloc[90:95].iterrows():
  
  print(index)
  dataset_name = row['title']['de']
    
  att_type = 'resources'
  val      = row[att_type]

  liste = checkDictionary(val)
    
  if len(liste) != 0:
    for elem in liste:
      completeList.append([dataset_name] + [att_type] + elem)


90
1c1afd74-493d-4c94-a35a-fd3a5bef9d15
Dictionary:
{'byte_size': 0, 'coverage': '', 'cache_last_updated': None, 'issued': '2017-12-12T00:00:00', 'package_id': '1c1afd74-493d-4c94-a35a-fd3a5bef9d15', 'datastore_contains_all_records_of_source_file': False, 'datastore_active': False, 'id': '435add39-0f46-4baa-b9d4-19cd330a2f18', 'size': None, 'license': '', 'display_name': {'fr': '', 'de': '', 'en': '', 'it': ''}, 'title': {'fr': '', 'de': '', 'en': '', 'it': ''}, 'download_url': '', 'state': 'active', 'media_type': '', 'hash': '', 'description': {'fr': '', 'de': '', 'en': '', 'it': ''}, 'format': 'SERVICE', 'mimetype_inner': None, 'url_type': None, 'mimetype': None, 'cache_url': None, 'name': {'fr': '', 'de': '', 'en': '', 'it': ''}, 'language': [], 'rights': 'NonCommercialAllowed-CommercialAllowed-ReferenceRequired', 'url': 'http://www.bfe.admin.ch/ogd37', 'created': '2017-12-12T16:18:37.119420', 'modified': '', 'last_modified': None, 'position': 0, 'revision_id': '9fce1239-3612-4fea-a

In [73]:
completeList

[['Schweizerische Holzenergiestatistik',
  'resources',
  'url',
  '435add39-0f46-4baa-b9d4-19cd330a2f18',
  'http://www.bfe.admin.ch/ogd37',
  'https://www.bfe.admin.ch/ogd37'],
 ['Schweizerische Statistik der erneuerbaren Energien',
  'resources',
  'url',
  '968a88c3-c434-4a2f-b4f3-4afd6ea4bf30',
  'http://www.bfe.admin.ch/ogd38',
  'https://www.bfe.admin.ch/ogd38'],
 ['Stand der Energiepolitik in den Kantonen',
  'resources',
  'url',
  '1178b595-b69a-44d6-b374-a754242c1ffc',
  'http://www.bfe.admin.ch/ogd39',
  'https://www.bfe.admin.ch/ogd39'],
 ['Strom-Marktpreis gemäss Art. 3 b bis, Abs. 2 und 3, EnV',
  'resources',
  'url',
  'f17a517c-0967-403a-b43b-f2aedd0407c1',
  'http://www.bfe.admin.ch/ogd42',
  'https://www.bfe.admin.ch/ogd42'],
 ['Strom-Marktpreis gemäss Art. 3 f, Abs. 3 EnV',
  'resources',
  'url',
  '683aef8f-93c7-4885-bba3-a84e8f4019c5',
  'http://www.bfe.admin.ch/ogd43',
  'https://www.bfe.admin.ch/ogd43']]

In [55]:

for index, row in df.iterrows():
  string = str(row['resources'])

  if 'http:' in string:
    print(index)
    print

20
23
24
55
56
63
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
completeList

[]

In [ ]:
check_url('https://data.geo.admin.ch/ch.bfe.solarenergie-eignung-daecher/solarenergie-eignung-daecher/solarenergie-eignung-daecher_2056_monthlydata.gpkg.zip')

KeyboardInterrupt: ignored

In [ ]:
requests.get('https://www.bfe.admin.ch/waerme-und-kaelteangebot')

<Response [200]>